In [2]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import xlsxwriter
from pathlib import Path

In [3]:
#File path to forecasting file
file_path = "./5QRF_template_Q4_23_Q4_24_v1.xlsx"

#Full list of Excel sheets
sheet_names_list = pd.ExcelFile(file_path).sheet_names

In [4]:
#List of countries for importing the country forecast
list_sheets = [ 'Canada',
 'Spain',
 'UK',
 'Turkey',
 'Latam',
 'Italy',
 'Rest',
 'France']

In [5]:
#Function to transform raw input of each Excel sheet into one dataframe

def create_forecast_df(country, file= file_path):
    
    data_import = pd.read_excel(file,header=1, index_col = "Unnamed: 1" , sheet_name=country)
    data_import["Country"] = country
    data_import = data_import[data_import.index.notnull()]
#cast numbers as Int
    data_import.iloc[:, 1:6] = data_import.iloc[:, 1:6].astype("int64")
#Reset index
    data_import = data_import.reset_index()
#Rename columns

    data_import = data_import.rename(columns={"Unnamed: 0": "Forecaster", "index": "Category"})
    
    excel_melted = data_import.melt(id_vars=["Category", "Forecaster", "Country"], value_vars=['Q4 2023', 'Q1 2024', 'Q2 2024', 'Q3 2024',
       'Q4 2024'], var_name = "Time" )
    excel_melted["Product"] = "Placeholder"
    excel_melted["Product"][excel_melted["Category"].str.contains("74", regex=False)] ="G360 CDx"
    excel_melted["Product"][excel_melted["Category"].str.contains("84", regex=False)] ="G360 Infinity"
    excel_melted["Product"][excel_melted["Category"].str.contains("Reveal", regex=False)] ="Reveal"
    excel_melted["Product"][excel_melted["Category"].str.contains("Shield", regex=False)] ="Shield"
    excel_melted["Forecast_category"] = "Placeholder"
    excel_melted["Forecast_category"][excel_melted["Category"].str.contains("volume", regex=False)] ="Volume"
    excel_melted["Forecast_category"][excel_melted["Category"].str.contains("Average", regex=False)] ="ASP"
    file_name = Path("./5QRF_template_Q4_23_Q4_24_v1.xlsx").stem.split("_")[2:4]
    excel_melted["5QRF_Starting_Quarter"] = "_".join(file_name)
    excel_melted= excel_melted[excel_melted["Forecast_category"] != "Placeholder"]
    excel_melted= excel_melted[excel_melted["Product"] != "Placeholder"]
    excel_melted["value"] = excel_melted["value"].astype("int64")
    excel_melted= excel_melted.pivot(index =['Forecaster', 'Country', 'Time', 'Product', '5QRF_Starting_Quarter'], columns = 'Forecast_category',values = "value")
    excel_melted = excel_melted.reset_index()
    return excel_melted



In [6]:
#Create empty dictionary
df_dict={}

for country in list_sheets:
    df_dict[country] = create_forecast_df(country,file= file_path )


In [7]:
#Concat all files
df = pd.concat([v for k,v in df_dict.items()])

In [11]:
df.to_csv("forecast_country.csv", index = False)